In [ ]:
#Imports - Update as project complexity increases
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix, RocCurveDisplay
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import randint
from pathlib import Path

In [ ]:
#Reading and looking at different aspects of data
data_path = Path("data") / "diabetes.csv"
df = pd.read_csv(data_path)
# Show top rows
df.head()
#We start to see strange values, it shouldn't be possible to have 0 skin thickness or 0 insulin. 
#Further investigation on these "false 0's" is needed

In [ ]:
#Create cleaning pipeline instead of inline code, allows for project modularity. Keep in mind this is only going to be used for EDA and data visualization, as I want to prevent potential data leakages. 

def cleaning_diabetes_data_for_eda(df):
    df = df.copy()
    zero_rows = {}
    columns_to_clean = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

    for col in columns_to_clean:
        #Chose to use median
        false_zero_indices = df.index[df[col] == 0].tolist()
        if false_zero_indices:
            zero_rows[col] = false_zero_indices
    print("False zeroes detected in the following columns:")
    for col, indices in zero_rows.items():
        print (f"-> Column '{col}' : Observations {indices}")
    df[columns_to_clean] = df[columns_to_clean].replace(0, np.nan)
    
    for col in columns_to_clean:
        df[col] = df[col].fillna(df[col].median())
    return df

clean_df = cleaning_diabetes_data_for_eda(df)
clean_df.head(5)

Part 2: 
Now that we've cleaned our dataset, we start using EDA to understand what features are going to be important in our final machine learning model. 

In [ ]:
clean_df.hist(figsize = (10,12), bins = 30, edgecolor = 'black') #Rule of thumb for bin specification
plt.tight_layout()
plt.show()
#Most plots seem to be right skewed or normal, mainly due to presense of 0. For example:
# A large number of women in our dataset have not been pregnant, or have only been pregnant once or twice. 
# Also, our median replacement creates large spikes at the median of our column for a lot of our predictors. It may be worth
# Looking into dropping Nan values instead of replacing, will look into on a later step in the process

In [ ]:
predictors = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction"]
for col in predictors:
    plt.figure(figsize = (6,4))
    sns.kdeplot(data = clean_df, x = col, hue = "Outcome", fill = True)
    plt.title(f"Distribution of {col} by Outcome")
    plt.tight_layout()
    plt.show()
#Here we see that pregnancies, glucose, skin thickness, insulin, and diabetes pedigree function have very different PDFs per outcome.
#Surprisingly, BMI and Blood pressure don't have extremely different PDFs per outcome in comparison to other predictors.
#However, all variables could be reasonably assumed to be influential on diabetes. More investigation is needed

In [ ]:
for col in predictors: 
    plt.figure(figsize = (6,4))
    sns.boxplot(x = "Outcome", y = col, data = clean_df)
    plt.title (f"{col} by Diabetes Outcome")
    plt.tight_layout()
    plt.show()

There are clear differences by outcome for BMI, Insulin, SkinThickness, BloodPressure, Glucose, and Pregnancies. Since we have relatively clear class seperation, a supervised model (labeled data) will work. No feature is completely dominant over the others, so a multivariable model will work best.

Modeling (Keep in mind we are now back to using the original dataset, as cleaning before splitting can lead to data leakage):

In [ ]:
#Step 0: Cleaning Transformer
class ZerotoNan(BaseEstimator, TransformerMixin): 
    "Replace false zeroes with Nans in specified columns"
    def __init__(self, columns): 
        self.columns = columns
    def fit(self, X, y = None):
        return self 
    def transform(self, X):
        X = X.copy()
        cols = [c for c in self.columns if c in X.columns]
        X[cols] = X[cols].replace(0, np.nan)
        return X
#Step 1: Initializing Models: 
models = { "Logistic Regression": LogisticRegression(random_state = 42, max_iter = 1000),
          "Random forest": RandomForestClassifier(random_state = 42),
          #Not linearly separable, so default kernel
          "Support Vector Machine": SVC(probability = True, random_state = 42)
         }
#Step 2: Splitting data before cleaning, to prevent data leakage
X = df.drop("Outcome", axis = 1)
y = df["Outcome"].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42, stratify = y)

results = []
fitted = {}

#Step 3: Applying 0 transformer, then imputing using median, scaling values, and applying classifier
for name, clf in models.items():
    pipe = Pipeline ([
        ("ZerotoNan", ZerotoNan(["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction"])),
        ('imputer', SimpleImputer(strategy = 'median')),
        ('scaler', StandardScaler()), 
        ('clf', clf)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1] if hasattr(pipe, "predict_proba") else None

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "ROC AUC Score": roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan,
    })
    fitted[name] = pipe
results_df = pd.DataFrame(results).sort_values(by = "ROC AUC Score", ascending = False).reset_index(drop = True)
print(results_df)
          

Looking at our Accuracy, Precision, Recall, and F1 scores, we're at 60% - 80%. Next, I'm going to work on model tuning through hyperparameter tuning, and a confusion matrix and ROC curve for the best model. 

In [ ]:
best_name = results_df.loc[0, "Model"]
best_model = fitted[best_name]
print(f"Best ROC AUC: {best_name}")
y_pred_best = best_model.predict(X_test)
y_proba_best = best_model.predict_proba(X_test)[:, 1] if hasattr(best_model, "predict_proba") else None
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best,zero_division = 0))
cm = confusion_matrix(y_test, y_pred_best)
fig, ax = plt.subplots(figsize=(5,4))
im = ax.imshow(cm, cmap="Blues") 
ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
ax.set_xticks([0,1]); ax.set_yticks([0,1])
ax.set_xticklabels(["No Diabetes","Diabetes"]); ax.set_yticklabels(["No Diabetes","Diabetes"])
for (i, j), val in np.ndenumerate(cm):
    ax.text(j, i, int(val), ha="center", va="center")
from pathlib import Path
Path("figures").mkdir(parents=True, exist_ok=True)
fig.savefig("figures/confusion_matrix.png", dpi=150, bbox_inches="tight") 
plt.show()

fig2, ax2 = plt.subplots(figsize=(5, 4))
RocCurveDisplay.from_estimator(best_model, X_test, y_test, ax = ax2)
plt.title(f"ROC Curve {best_name}")
fig2.savefig("figures/roc_curve.png", dpi=150, bbox_inches="tight")
plt.show()


Hyperparameter Tuning

In [ ]:
rf_pipe = Pipeline ([
        ("ZerotoNan", ZerotoNan(["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction"])),
        ('imputer', SimpleImputer(strategy = 'median')),
        ('scaler', StandardScaler()), 
        ('clf', RandomForestClassifier(random_state = 42, class_weight = "balanced"))
    ])
param_dist = {
    "clf__n_estimators": randint(100,600),
    "clf__max_depth": randint(2,20),
    "clf__min_samples_split": randint(2,20),
    "clf__min_samples_leaf": randint(1,10),
    "clf__max_features": ["sqrt","log2", None],
}
cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
rf_search = RandomizedSearchCV(
    rf_pipe,
    param_distributions=param_dist,
    n_iter=40,
    scoring="roc_auc",
    n_jobs= -1,
    cv=cv,
    random_state=42,
    verbose=1,
)
rf_search.fit(X_train, y_train)
rf_best = rf_search.best_estimator_
print("Best Random Forest Parameters:", rf_search.best_params_)
print("Best CV ROC AUC:", rf_search.best_score_)

y_pred_tuned = rf_best.predict(X_test)
y_proba_tuned = rf_best.predict_proba(X_test)[:,1]

tuned_metrics = {
    "Model": "Random Forest (Tuned)",
    "Accuracy": accuracy_score(y_test, y_pred_tuned),
    "Precision": precision_score(y_test, y_pred_tuned, zero_division=0),
    "Recall": recall_score(y_test, y_pred_tuned, zero_division=0),
    "F1 Score": f1_score(y_test, y_pred_tuned, zero_division=0),
    "ROC AUC Score": roc_auc_score(y_test, y_proba_tuned),
}
compare_df = (pd.concat([results_df, pd.DataFrame([tuned_metrics])], ignore_index=True)
                .sort_values(by="ROC AUC Score", ascending=False)
                .reset_index(drop=True))
print(compare_df)

Conclusion: 
Our Random Forest Tuned Model ended up being the best overall. In screening situations, recall is typically prioritized, as we want to capture as many true positives as possible. 
In terms of our test metrics, we have an accuracy of 74%, precision of 60%, recall of 74%, f1 score of 66%, and a ROC AUC of 82%.
Also, since our CV best AUC is 84%, and our test AUC is 82%, we are relatively close, meaning we do not have overfitting. 